<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/Rodada1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação de bibliotecas

In [ ]:
#Importing needed libraries
!pip install -U scikit-learn

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression



from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


!pip install -U spacy
!python -m spacy download pt
import spacy



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Dataset Hate BR**

In [4]:
from pandas.core.groupby import base
base_hatebr = pd.read_csv('/content/drive/MyDrive/PIC /Dataset/HateBR.csv')

In [5]:
base_hatebr.offensive_language.value_counts()

1    3500
0    3500
Name: offensive_language, dtype: int64

Dividindo em base de teste e base de treinamento


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.instagram_comments,
    base_hatebr.offensive_language,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [7]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


In [8]:
y_train.value_counts()

1    2800
0    2800
Name: offensive_language, dtype: int64

In [9]:
y_test.value_counts()

1    700
0    700
Name: offensive_language, dtype: int64

**TREINAMENTOS** sem pré-processamento

Treinamento com Naive Bayes


In [10]:
#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       700
           1       0.82      0.86      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Support Vector Machine(SVM)

In [11]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.85      0.84       700
           1       0.85      0.84      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Multilayer Perceptron(MLP)

In [12]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       700
           1       0.85      0.83      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Regressão Logística(LR)

In [13]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       700
           1       0.82      0.83      0.83       700

    accuracy                           0.83      1400
   macro avg       0.83      0.83      0.83      1400
weighted avg       0.83      0.83      0.83      1400



**TREINAMENTOS** com pré-processamento

Função de pré-processamento dos dados


*   Foi notado que ao realizar o procedimento de deixar somente letras o modelo teve piora de pontuação F1, sem a função o texto continua com emojis e caracteres especiais...


In [14]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)

Criando uma nova coluna com o texto com pré-processamento

In [15]:
base_hatebr['preprocessed_comments'] =  base_hatebr['instagram_comments'].apply(preprocess)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.	preprocessed_comments,
    base_hatebr.offensive_language,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [17]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


Naive Bayes

In [18]:
#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.86      0.84      0.85       700
           1       0.84      0.86      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report #comparação entre os resultados
accuracy_score(y_test, y_pred) #retorna a porcentagem dos acertos

0.8492857142857143

Support Vector Machine(SVM)

In [20]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC(C=2.0, kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(C=2.0, kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



Multilayer Perceptron(MLP)

In [21]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       700
           1       0.87      0.81      0.84       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



Regressão Logística(LR)

In [22]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



**TOLD BR**

In [24]:
told_br = pd.read_csv("/content/drive/MyDrive/PIC /Dataset/ToLD-BR_alpha.csv")

Tranformação em base binária

In [26]:
told_br["toxic"] = 0
told_br["toxic_1"] = 0
told_br["toxic_2"] = 0
told_br["toxic_3"] = 0
told_br["homophobia"] = 0
told_br["obscene"] = 0
told_br["insult"] = 0
told_br["racism"] = 0
told_br["misogyny"] = 0
told_br["xenophobia"] = 0
for i, row in told_br.iterrows():
    count = 0
    if 1 in list(row[["homophobia_1", "obscene_1", "insult_1", "racism_1", "misogyny_1", "xenophobia_1"]]):
        told_br.loc[i, "toxic_1"] = 1
        count += 1
    if 1 in list(row[["homophobia_2", "obscene_2", "insult_2", "racism_2", "misogyny_2", "xenophobia_2"]]):
        told_br.loc[i, "toxic_2"] = 1
        count += 1
    if 1 in list(row[["homophobia_3", "obscene_3", "insult_3", "racism_3", "misogyny_3", "xenophobia_3"]]):
        told_br.loc[i, "toxic_3"] = 1
        count += 1

    told_br.loc[i, "homophobia"] = np.sum(list(row[["homophobia_1", "homophobia_2", "homophobia_3"]]))

    told_br.loc[i, "obscene"] = np.sum(list(row[["obscene_1", "obscene_2", "obscene_3"]]))

    told_br.loc[i, "insult"] = np.sum(list(row[["insult_1", "insult_2", "insult_3"]]))

    told_br.loc[i, "racism"] = np.sum(list(row[["racism_1", "racism_2", "racism_3"]]))

    told_br.loc[i, "misogyny"] = np.sum(list(row[["misogyny_1", "misogyny_2", "misogyny_3"]]))

    told_br.loc[i, "xenophobia"] = np.sum(list(row[["xenophobia_1", "xenophobia_2", "xenophobia_3"]]))

    if count >= 1:
        told_br.loc[i, "toxic"] = 1

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    told_br.text,
    told_br.toxic,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = told_br.toxic
)

**TREINAMENTOS** sem pré-processamento

Naive Bayes

In [28]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.68      0.87      0.76      2349
           1       0.74      0.47      0.58      1851

    accuracy                           0.69      4200
   macro avg       0.71      0.67      0.67      4200
weighted avg       0.70      0.69      0.68      4200



Support Vector Machine(SVM)

In [29]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.77      0.80      0.78      2349
           1       0.73      0.69      0.71      1851

    accuracy                           0.75      4200
   macro avg       0.75      0.75      0.75      4200
weighted avg       0.75      0.75      0.75      4200



Multilayer Perceptron(MLP)

In [30]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.73      0.73      0.73      2349
           1       0.66      0.66      0.66      1851

    accuracy                           0.70      4200
   macro avg       0.70      0.70      0.70      4200
weighted avg       0.70      0.70      0.70      4200



 Regressão Logística(LR)

In [31]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.75      0.83      0.79      2349
           1       0.75      0.65      0.69      1851

    accuracy                           0.75      4200
   macro avg       0.75      0.74      0.74      4200
weighted avg       0.75      0.75      0.75      4200



**TREINAMENTOS** com pré-processamento

In [32]:
# print(preprocess(base_hatebr['instagram_comments']))
told_br['preprocessed_text'] =  told_br['text'].apply(preprocess)

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    told_br.preprocessed_text,
    told_br.toxic,
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = told_br.toxic
)

Naive Bayes

In [34]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.70      0.84      0.76      2349
           1       0.73      0.53      0.61      1851

    accuracy                           0.71      4200
   macro avg       0.71      0.69      0.69      4200
weighted avg       0.71      0.71      0.70      4200



Support Vector Machine(SVM)

In [35]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      2349
           1       0.72      0.67      0.69      1851

    accuracy                           0.74      4200
   macro avg       0.74      0.73      0.73      4200
weighted avg       0.74      0.74      0.74      4200



Multilayer Perceptron(MLP)

In [36]:
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.72      0.75      0.74      2349
           1       0.67      0.63      0.65      1851

    accuracy                           0.70      4200
   macro avg       0.70      0.69      0.69      4200
weighted avg       0.70      0.70      0.70      4200



 Regressão Logística(LR)

In [37]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.74      0.84      0.79      2349
           1       0.75      0.63      0.68      1851

    accuracy                           0.74      4200
   macro avg       0.75      0.73      0.74      4200
weighted avg       0.75      0.74      0.74      4200

